In [1]:
import numpy as np
import pycram

Unknown attribute "type" in /robot[@name='pr2']/link[@name='base_laser_link']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='wide_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='narrow_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='laser_tilt_link']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='base_laser_link']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='wide_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='narrow_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='laser_tilt_link']


In [2]:
from pycram.bullet_world import BulletWorld, Object

world = BulletWorld()
kitchen = Object("kitchen", "environment", "kitchen.urdf")

Unknown tag "material" in /robot[@name='plane']/link[@name='planeLink']/collision[1]
Unknown tag "contact" in /robot[@name='plane']/link[@name='planeLink']
Unknown tag "material" in /robot[@name='plane']/link[@name='planeLink']/collision[1]
Unknown tag "contact" in /robot[@name='plane']/link[@name='planeLink']
Scalar element defined multiple times: limit
Scalar element defined multiple times: limit


In [3]:
import pycram.bullet_world_reasoning as btr

milk = Object("milk", "milk", "milk.stl", position=[1, 0, 1])
pr2 = Object("pr2", "robot", "pr2.urdf")
bowl = Object("bowl", "bowl", "bowl.stl", position=[1, 0.5, 1])

Unknown attribute "type" in /robot[@name='pr2']/link[@name='base_laser_link']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='wide_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='narrow_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='laser_tilt_link']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='base_laser_link']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='wide_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='narrow_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='laser_tilt_link']


# Remarks to Pick and Place Plan
1. The milk pack is placed on top of the bowl, since the SemanticCostmapLocation returns a single identical pose for both objects
2. Is there any reasoner to check if two or more objects clip into each other ?
3. How the hell do we implement pouring, we probably have to implement our own pouring motion.

In [36]:
from pycram.designators import *
from pycram.process_module import simulated_robot
from pycram.enums import Arms

#for x in range(20):

world.reset_bullet_world()

milk_desig = BelieveObject(names=["milk"])
bowl_desig = BelieveObject(names=["bowl"])
robot_desig = BelieveObject(types=["robot"]).resolve()
# pycram.designators.motion_designator.MoveTCPMotion()
for believe_obj in [bowl_desig, milk_desig]:
    with simulated_robot:
        ParkArmsAction([Arms.BOTH]).resolve().perform()
        MoveTorsoAction([0.05]).resolve().perform()
        LookAtAction(targets=[milk_desig.resolve().pose]).resolve().perform()
        obj_desig = DetectAction(believe_obj).resolve().perform()
        location_description = CostmapLocation(target=obj_desig, reachable_for=robot_desig)
        location_description.resolve()
        for pose in location_description:
            p = pose.pose
            reachable_arms = pose.reachable_arms
            break
        NavigateAction.Action(p).perform()
        PickUpAction(object_designator_description=believe_obj,
                         arms=reachable_arms,
                         grasps=["front"]).resolve().perform()
        ParkArmsAction([Arms.BOTH]).resolve().perform()
        kitchen_description = ObjectDesignatorDescription(names=['kitchen']).resolve()
        object_locations = SemanticCostmapLocation(urdf_link_name="kitchen_island_surface", part_of=kitchen_description,
                                                   for_object=obj_desig).resolve()
        place_location = CostmapLocation(target=object_locations.pose, reachable_for=robot_desig, reachable_arm=reachable_arms[0])
        place_location.resolve()

        for pose in place_location:
            p3 = pose.pose
            break

        NavigateAction.Action(p3).perform()
        PlaceAction(object_designator_description=believe_obj,
                    target_locations=[object_locations.pose],
                    arms=["left"]).resolve().perform()
        ParkArmsAction.Action(Arms.BOTH).perform()

        # Testing tilting motion ?
        position = list(pr2.get_link_position("l_gripper_tool_frame"))
        orientation = list(pr2.get_link_orientation("l_gripper_tool_frame"))
        orientation[0] = orientation[0] + 0.4#
        position[0] = position[0] + 0.4
        MoveTorsoAction([0.25]).resolve().perform()
        MoveTCPMotion(target=[list(position), list(orientation)], arm="left").resolve().perform()

In [ ]:
"""
Benutze stattdessen den pose_generator um Posen für die Milchpackung zu generieren
"""

from pycram.enums import Arms
from pycram.designators.action_designator import ParkArmsAction
import pycram.costmaps as cm
from pycram.process_module import simulated_robot, with_simulated_robot

s = cm.SemanticCostmap(kitchen, "kitchen_island_surface", size=100, resolution=0.02)
milk_aabb = milk.get_AABB()
# Sample 1000 poses on the kitchen island surface
poses = pycram.pose_generator_and_validator.pose_generator(costmap=s, number_of_samples=1000)
with simulated_robot:
    ParkArmsAction([Arms.BOTH]).resolve().perform()

#Todo: Fix collision and clipping through the walls.
for pose in poses:
    print(pose)
    # Without this offset, the milkbottle clips into the islands surface
    offset = np.sqrt(np.square(pose[0][2] - milk_aabb[0][2]))
    position = np.array([pose[0][0],pose[0][1],pose[0][2] + offset])
    milk.set_position(position)
    robot_desig = BelieveObject(types=["robot"]).resolve()
    location_description = CostmapLocation(target=milk_desig, reachable_for=robot_desig)
    for pose in location_description:
        p = pose.pose
        pr2.set_position_and_orientation(p[0],p[1])
        # Is object visible for the pr2
        is_visible = CostmapLocation(target=milk_desig, visible_for=robot_desig)
        break

In [83]:
world.reset_bullet_world()

In [85]:
world.exit()